<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/Skipgram_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

In [2]:
import csv
import pandas as pd

# Replace 'data.tsv' with the path to your TSV file
tsv_file = '/content/drive/MyDrive/강이지_옷/강아지옷.tsv'

# Initialize an empty list to store the data from the TSV file
data = []

# Read the TSV file and store its content in the 'data' list
with open(tsv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.append(row)

df = pd.DataFrame(data[1:], columns=data[0])

df = df[:len(df)//100]

df

,user_id,score,date,review,is_month,is_repurch
0,ur****,4,23.03.20.,군더더기 없어요 괜찮아요,False,False
1,blac*****,5,23.02.05.,촉감이 좋아서 잘입어요,True,False
2,minw******,5,22.12.12.,흰색 겉옷과 같이 입히려고 샀는데 전에 이곳에서 흰색도 샀었더라구요-\n흰색은 가디...,False,False
3,blac*****,5,23.02.05.,좋아요~~~~~~~~,True,False
4,dush******,5,22.12.10.,색상도 이쁘고 신축성도 좋네요,False,False
...,...,...,...,...,...,...
1277,suge*******,5,20.05.26.,두장사려고해서 노랑이랑 뭐살지고민하다가 그나마 이게낫겟다햇는데 노랑보다 이게더이뻐요...,False,False
1278,ilov****,5,20.06.24.,색도 예쁘고 좋아요!,False,False
1279,3025***,5,20.06.22.,사이즈 딱맞고 너무 귀여워용 ㅎㅎ,False,False
1280,yybi****,5,20.05.30.,너무이뻐요 사이즈도 딱이고 여름이라 더울것같아 털을 짧게 밀긴햇지만 자외선때문에 걱...,False,False


In [3]:
# 말뭉치 생성
corpus = "-".join(df['review'].tolist())
corpus

"군더더기 없어요 괜찮아요-촉감이 좋아서 잘입어요-흰색 겉옷과 같이 입히려고 샀는데 전에 이곳에서 흰색도 샀었더라구요-\n흰색은 가디건과 함께 입히니 귀엽더라구요-\n나중에 봐서 분홍색도 사려구요-\n하나만 입혀도 엄청 귀여워요--좋아요~~~~~~~~-색상도 이쁘고 신축성도 좋네요-레이어드용으로 딱이네요-너무이뻐요 색상도 디자인도-겨울이라 패딩안에 입히려구 샀는데 사이즈도 그렇고 두께도 너무 적당합니다! ㅎㅎ-불편해 하지 않고, 너무 예쁘네요.-색이쁘고 좋아용 굿-티셔츠너무이뻐요 재구매하려구요-길이가 좀 짧은감이 있어요-아이보리로 구매햇어요~!! 어떤 코디에나 무난히 어울려 귀엽다는말 많이들엇네요!\n저는 저희 아기 옷구매 멍블링밖에안해요 ㅎㅎ\n이너로 입기 좋습니다! 가디건이랑 같이구매햇는데 \n아기가 슬개구 수술해서 후에 인증샷올릴게요!-색감이 사진보다 더 탁한 소라색이에요-길이감이 짧아요~~~-좋아요 좋아요 좋아요 좋아요 좋아요-예전에 샀던 아가일 조끼안에 입기 좋은 베이직한 티셔츠에요ㅎㅎ 카라형이라 좋고, 재질도 좋아요~^^ 감사합니다 : )-완전맘에들어요 카라부분 너무이쁩니다 바느질도 깔끔해요-1킬로가 채안되다보니 제일작은사이즈인데도\n크네요~ 등길이는대충 맞아요\n어깨부분 수선해서 입혔더니 좀 나아요 ㅎ\n입고도 잘자네요 이뻐요~-우리집 옷주인이랑 찰떡이네요 너무 귀여워요-이쁘게 잘입고있어요-귀엽고 이쁘고 다합니다-예뻐요<~~~~~~~~~~~~~~~~~~~-예뻐요~~~~~~~~~~~~~~~~~~~~-원피스 입힐 때 이너로 입혀줘요 어디에도 잘 어울리고 애기가 잘 입어서 좋은 것 가타요-면이 넘 좋네요 옷을 4벌을 주문했는데 3벌이 배송중이라고 뜨길래 연락을 했더니 이옷이 이염이있어 3벌먼저 보냈다고 해서 한참있다 오겠거니 했는데 그뒷날 보냈더라구요 뒷날 한꺼번에 보내도 되는데 기다리고 있는 고객의 입장 먼저 보내셨더라구요 섬세한 배려에 감동 받았어요 그런데 옷이 왜이리 부드러워요 자주 입을것같아요 기본티 이지만 센스도 있고 면좋고 짱 좋네요~ 감사해

In [4]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣 -]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용
    return result

In [5]:
corpus = apply_regular_expression(corpus)

corpus

'군더더기 없어요 괜찮아요-촉감이 좋아서 잘입어요-흰색 겉옷과 같이 입히려고 샀는데 전에 이곳에서 흰색도 샀었더라구요-흰색은 가디건과 함께 입히니 귀엽더라구요-나중에 봐서 분홍색도 사려구요-하나만 입혀도 엄청 귀여워요--좋아요-색상도 이쁘고 신축성도 좋네요-레이어드용으로 딱이네요-너무이뻐요 색상도 디자인도-겨울이라 패딩안에 입히려구 샀는데 사이즈도 그렇고 두께도 너무 적당합니다 ㅎㅎ-불편해 하지 않고 너무 예쁘네요-색이쁘고 좋아용 굿-티셔츠너무이뻐요 재구매하려구요-길이가 좀 짧은감이 있어요-아이보리로 구매햇어요 어떤 코디에나 무난히 어울려 귀엽다는말 많이들엇네요저는 저희 아기 옷구매 멍블링밖에안해요 ㅎㅎ이너로 입기 좋습니다 가디건이랑 같이구매햇는데 아기가 슬개구 수술해서 후에 인증샷올릴게요-색감이 사진보다 더 탁한 소라색이에요-길이감이 짧아요-좋아요 좋아요 좋아요 좋아요 좋아요-예전에 샀던 아가일 조끼안에 입기 좋은 베이직한 티셔츠에요ㅎㅎ 카라형이라 좋고 재질도 좋아요 감사합니다  -완전맘에들어요 카라부분 너무이쁩니다 바느질도 깔끔해요-킬로가 채안되다보니 제일작은사이즈인데도크네요 등길이는대충 맞아요어깨부분 수선해서 입혔더니 좀 나아요 ㅎ입고도 잘자네요 이뻐요-우리집 옷주인이랑 찰떡이네요 너무 귀여워요-이쁘게 잘입고있어요-귀엽고 이쁘고 다합니다-예뻐요-예뻐요-원피스 입힐 때 이너로 입혀줘요 어디에도 잘 어울리고 애기가 잘 입어서 좋은 것 가타요-면이 넘 좋네요 옷을 벌을 주문했는데 벌이 배송중이라고 뜨길래 연락을 했더니 이옷이 이염이있어 벌먼저 보냈다고 해서 한참있다 오겠거니 했는데 그뒷날 보냈더라구요 뒷날 한꺼번에 보내도 되는데 기다리고 있는 고객의 입장 먼저 보내셨더라구요 섬세한 배려에 감동 받았어요 그런데 옷이 왜이리 부드러워요 자주 입을것같아요 기본티 이지만 센스도 있고 면좋고 짱 좋네요 감사해요-일단 너무 귀엽구요 그냥 귀여워요 넘 예쁘고 귀여워요-샀던 옷 중에 제일 예쁘고 사이즈도 적당한건 처음 인거 같아요 ㅜㅜ-안에 받쳐입히기 좋아요-이너티로 딱 좋

In [6]:
stop_words = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stop_words[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [7]:
import numpy as np
import string
from nltk.corpus import stopwords

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

class word2vec(object):
    def __init__(self):
        self.N = 10
        self.X_train = []
        self.y_train = []
        self.window_size = 2
        self.alpha = 0.001
        self.words = []
        self.word_index = {}

    def initialize(self,V,data):
        self.V = V
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N))
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V))

        self.words = data
        for i in range(len(data)):
            self.word_index[data[i]] = i


    def feed_forward(self,X):
        self.h = np.dot(self.W.T,X).reshape(self.N,1)
        self.u = np.dot(self.W1.T,self.h)
        #print(self.u)
        self.y = softmax(self.u)
        return self.y

    def backpropagate(self,x,t):
        e = self.y - np.asarray(t).reshape(self.V,1)
        # e.shape is V x 1
        dLdW1 = np.dot(self.h,e.T)
        X = np.array(x).reshape(self.V,1)
        dLdW = np.dot(X, np.dot(self.W1,e).T)
        self.W1 = self.W1 - self.alpha*dLdW1
        self.W = self.W - self.alpha*dLdW

    def train(self,epochs):
        for x in range(1,epochs):
            self.loss = 0
            for j in range(len(self.X_train)):
                self.feed_forward(self.X_train[j])
                self.backpropagate(self.X_train[j],self.y_train[j])
                C = 0
                for m in range(self.V):
                    if(self.y_train[j][m]):
                        self.loss += -1*self.u[m][0]
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u)))
            print("epoch ",x, " loss = ",self.loss)
            self.alpha *= 1/( (1+self.alpha*x) )

    def predict(self,word,number_of_predictions):
        if word in self.words:
            index = self.word_index[word]
            X = [0 for i in range(self.V)]
            X[index] = 1
            prediction = self.feed_forward(X)
            output = {}
            for i in range(self.V):
                output[prediction[i][0]] = i

            top_context_words = []
            for k in sorted(output,reverse=True):
                top_context_words.append(self.words[output[k]])
                if(len(top_context_words)>=number_of_predictions):
                    break

            return top_context_words
        else:
            print("Word not found in dictionary")

In [8]:
from konlpy.tag import Okt
import string

def preprocessing(corpus):
    okt = Okt()
    training_data = []
    sentences = corpus.split("-")

    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip()
        words_and_tags = okt.pos(sentences[i])

        filtered_words = [word for word, tag in words_and_tags
                          if tag in ['Noun', 'Verb', 'Adjective']
                          and word not in stop_words]

        x = [word.strip(string.punctuation) for word in filtered_words]
        x = [word for word in filtered_words if len(word) > 1]
        training_data.append(x)

    return training_data


def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i

    #for i in range(len(words)):
    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]

            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(V,data)

    return w2v.X_train,w2v.y_train

In [10]:
epochs = 10

training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs)

epoch  1  loss =  201339.18962312443
epoch  2  loss =  201090.8970415834
epoch  3  loss =  200850.24998964113
epoch  4  loss =  200616.08783324237
epoch  5  loss =  200387.31471917746
epoch  6  loss =  200162.88867592337
epoch  7  loss =  199941.81197261458
epoch  8  loss =  199723.1227668968
epoch  9  loss =  199505.88808089073


In [28]:
print(w2v.predict("강아지",10))

['겨울', '감사합니다', '툐조어툐', '걸치니', '와전', '불편해하는데', '귀엽긴', '덮게', '여고하네요', '면서']


In [17]:
training_data

[['군더더기', '없어요', '괜찮아요'],
 ['촉감', '좋아서', '입어요'],
 ['흰색', '겉옷', '입히려고', '샀는데', '흰색', '샀었더라구요'],
 ['흰색', '가디건', '입히니', '귀엽더라구요'],
 ['나중', '봐서', '분홍색', '사려구요'],
 ['하나', '입혀도', '귀여워요'],
 [],
 ['좋아요'],
 ['색상', '이쁘고', '신축', '성도', '좋네요'],
 ['레이어', '드용', '딱이네요'],
 ['이뻐요', '색상', '디자인'],
 ['겨울', '패딩안', '입히려', '샀는데', '사이즈', '그렇고', '두께', '적당합니다'],
 ['불편해', '하지', '않고', '예쁘네요'],
 ['이쁘고', '좋아용'],
 ['티셔츠', '이뻐요', '재구매', '하려구요'],
 ['길이', '짧은', '있어요'],
 ['아이보리',
  '어떤',
  '코디',
  '무난',
  '어울려',
  '귀엽다는말',
  '들엇네',
  '저희',
  '아기',
  '구매',
  '멍블링',
  '안해',
  '입기',
  '좋습니다',
  '가디건',
  '구매',
  '는데',
  '아기',
  '개구',
  '수술',
  '해서',
  '인증샷',
  '올릴게요'],
 ['색감', '사진', '탁한', '소라색'],
 ['길이', '짧아요'],
 ['좋아요', '좋아요', '좋아요', '좋아요', '좋아요'],
 ['예전',
  '샀던',
  '아가',
  '조끼',
  '입기',
  '좋은',
  '베이직',
  '티셔츠',
  '카라',
  '좋고',
  '재질',
  '좋아요',
  '감사합니다'],
 ['완전', '들어요', '카라', '부분', '이쁩니다', '바느질', '깔끔해요'],
 ['킬로',
  '안되다',
  '보니',
  '제일',
  '작은',
  '사이즈',
  '도크',
  '길이',
  '대충',
  '맞아요',
  '어깨',
  '부분',
  '수선해서',
  '입혔더니',